## Precarga de librerias y funciones

In [ ]:
%run "../../recurrentes.ipynb"

In [ ]:
%run "../../funciones.ipynb"

#### Excel

In [ ]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l
# d) dataset_10_meses_l

dataset = dataset_10_meses
dataset_name = os.path.basename(dataset)
df_train = pd.read_parquet(dataset)


In [ ]:
import umap
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
df_train['foto_mes'].shape

In [ ]:
df_train.shape

In [ ]:
numero_de_cliente = df_train['numero_de_cliente']
caracteristicas = df_train.drop(['numero_de_cliente','clase_ternaria'], axis=1)
caracteristicas_sin_nulos = caracteristicas.dropna(axis=1)

In [ ]:
caracteristicas_sin_nulos.shape

In [ ]:

scaler = StandardScaler()
caracteristicas_escaladas = scaler.fit_transform(caracteristicas_sin_nulos)

# 4. Aplicar UMAP
reductor = umap.UMAP(n_components=2, n_neighbors=10, n_jobs=6)
embedding = reductor.fit_transform(caracteristicas_escaladas)

# 5. Crear DataFrame de embeddings con 'numero_de_cliente'
embedding_df = pd.DataFrame(embedding, columns=['UMAP1', 'UMAP2'])
df_umap = pd.concat([numero_de_cliente.reset_index(drop=True), embedding_df], axis=1)




In [ ]:
df_umap.to_csv('embedding_umap_con_cliente.csv', index=False)

## Punto de corte

In [ ]:
merged_data = prediccion.merge(bajas, on='numero_de_cliente', how='inner')
        
# Ordenar por probabilidad de mayor a menor
merged_data_sorted = merged_data.sort_values('Probabilidad', ascending=False)

# Seleccionar top clientes
top_clients = merged_data_sorted.iloc[:12000].copy()  # Trabajar con una copia

# Convertir 'clase_ternaria' a variable binaria para los top clientes
top_clients.loc[:, 'bajas_reales'] = (top_clients['clase_ternaria'] == 'BAJA+2').astype(int)

top_clients.value_counts('clase_ternaria')

In [ ]:
def calculoGanancia(bajas,prediccion,corte):
    ''' 
    Calcula la ganancia para una semilla específica.
    
    Parámetros:
    bajas: DataFrame con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: DataFrame con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    random_state: int, semilla para train_test_split.
    
    Retorna:
    ganancia_publico: Ganancia para el público.
    ganancia_privado: Ganancia para el privado.
    '''
    # Realizar el split en público y privado
    Publico, Privado = train_test_split(
        bajas,
        test_size=0.7,
        stratify=bajas['clase_ternaria'],
        random_state=123
    )

    # Clientes que decido estimular
    estimulos = prediccion.iloc[:corte] 

    # Obtener los estímulos en el conjunto público y privado
    estimulos_publico = pd.merge(estimulos, Publico, on='numero_de_cliente', how='inner')
    estimulos_privado = pd.merge(estimulos, Privado, on='numero_de_cliente', how='inner')

    # Calcular los verdaderos positivos en cada conjunto
    TP_publico = estimulos_publico[estimulos_publico['clase_ternaria'] == 'BAJA+2']
    TP_privado = estimulos_privado[estimulos_privado['clase_ternaria'] == 'BAJA+2']

    # 5. Calcular la ganancia para cada conjunto con normalización
    # Primero, calculamos la ganancia en cada conjunto
    ganancia_publico_sin_norm = (len(TP_publico) * 273000) - ((len(estimulos_publico) - len(TP_publico)) * 7000)
    ganancia_privado_sin_norm = (len(TP_privado) * 273000) - ((len(estimulos_privado) - len(TP_privado)) * 7000)

    # Luego, normalizamos dividiendo por el porcentaje correspondiente
    ganancia_publico = ganancia_publico_sin_norm / 0.3
    ganancia_privado = ganancia_privado_sin_norm / 0.7

    return ganancia_publico, ganancia_privado


In [ ]:
inicio = time.time()

resultados = []
cortes = range(5000, 20000, 1000)


pred_model_sorted = prediccion.sort_values('Probabilidad', ascending=False)
model_name = 'LightGBM'

# Iteramos sobre cada corte
for corte in cortes:
    ganancia_publico, ganancia_privado = calculoGanancia(bajas, pred_model_sorted, corte)
    
    # Almacenamos los resultados
    resultados.append({
        'Modelo': model_name,
        'Corte': corte,
        'Ganancia Público': ganancia_publico,
        'Ganancia Privado': ganancia_privado
    })

# Convertimos los resultados en un DataFrame
resultados = pd.DataFrame(resultados)

fin = time.time()
tiempo_ejecucion = fin - inicio

print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")

In [ ]:
# Pivotamos el DataFrame 'resultados' para reorganizar las ganancias
resultados_pivot = resultados.pivot_table(
    index='Corte',
    columns='Modelo',
    values=['Ganancia Público', 'Ganancia Privado']
)

# Aplanamos las columnas para facilitar el acceso
resultados_pivot.columns = [f'{ganancia}_{modelo}' for ganancia, modelo in resultados_pivot.columns]

# Reordenamos las columnas alternando 'Público' y 'Privado' para cada modelo
# Ordenamos primero por el modelo, luego alternando entre 'Público' y 'Privado'
columnas_ordenadas = []
for modelo in resultados['Modelo'].unique():
    columnas_ordenadas.append(f'Ganancia Público_{modelo}')
    columnas_ordenadas.append(f'Ganancia Privado_{modelo}')

# Reorganizamos el DataFrame usando el nuevo orden de columnas
resultados_pivot = resultados_pivot[columnas_ordenadas]

# Convertimos el índice 'Corte' en una columna si prefieres tenerla como tal
resultados_pivot = resultados_pivot.reset_index()
resultados_pivot